### Compare to the results from Royden_Husson_06_gji

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.RoydenHusson06 as RH06

In [ ]:
yr = 365.0 * 24.0 * 3600.0
cm_per_yr = 1e-2 / yr

Vt = 1.0 * cm_per_yr
Vm = 1.0 * cm_per_yr
Vr = 1.0 * cm_per_yr
L = 1000e3
a = L / 2.0
lbd = 700e3
mu = 1e20
n_hs = 100

Vx_dzs_nd_std = np.ones(n_hs)

Hele_Shaw = RH06.HELE_SHAW(Vt, Vm, Vr, L, lbd, mu, n_hs)
As = 0.001 * np.ones(n_hs) * mu / lbd**2.0 * (Vt + Vm) # initial guess
Hele_Shaw.AssignA(As)

In [ ]:
# plot the result with the initial guess
x = 10e3 # 10 km
ym = L
fig, ax = plt.subplots()
Hele_Shaw.PlotVxdz(x, ym, ax=ax)

In [ ]:
x = 10e3 # 10 km
n_iter = 10
Vxdzs = Hele_Shaw.Vxdz_array(x)
Vx_dzs_nd = Vxdzs / lbd / Vr

for i in range(n_iter):
    J = Hele_Shaw.Jocobi(x)
    J1 = np.dot(J, J.transpose())
    J1_inv = np.linalg.inv(J1)
    # print(J)
    # print(J1)
    
    # print(As)
    
    d_As = np.dot(J1_inv, np.dot(J, Vx_dzs_nd_std - Vx_dzs_nd))
    # print(d_As) # debug
    As += d_As
    # print(As_new)
    
    Hele_Shaw.AssignA(As)
    Vxdzs = Hele_Shaw.Vxdz_array(x)
    Vx_dzs_nd = Vxdzs / lbd / Vr

In [ ]:
# plot the result with the initial guess
x = 10e3 # 10 km
ym = L
fig, ax = plt.subplots()
Hele_Shaw.PlotVxdz(x, ym, ax=ax)